In [16]:
from __future__ import print_function
import matplotlib.pyplot as plt
%matplotlib inline

import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import numpy as np
from models.resnet import ResNet
from models.unet import UNet
from models.skip import skip
from models.dcgan import dcgan
import torch
import torch.optim
from sklearn.model_selection import train_test_split
from torchmetrics.image import StructuralSimilarityIndexMeasure
from torchmetrics.image import TotalVariation
import albumentations as A
from albumentations.pytorch import ToTensorV2
# from utils.inpainting_utils import *
import torch.nn as nn
from PIL import Image

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark =True
dtype = torch.cuda.FloatTensor

PLOT = True
imsize = -1
dim_div_by = 64

In [17]:
import os
import sys
from os import listdir
import pandas as pd
import numpy as np
import glob
import cv2
import json
from os.path import expanduser
import splitfolders
import shutil
# from define_path import Def_Path

from tqdm import tqdm

import torch 
import torchvision
from torchvision import models
from torchvision.models.detection.rpn import AnchorGenerator
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn 
import torchvision.transforms as T
from torchvision.transforms import functional as F
# from torchsummary import summary

from sklearn.model_selection import train_test_split

import albumentations as A # Library for augmentations

import matplotlib.pyplot as plt 
from PIL import Image

# # Get the directory of the current script
# current_script_dir = os.path.dirname(os.path.abspath(__file__))

# # Navigate up to the 'scripts' directory
# scripts_dir = os.path.dirname(current_script_dir)

# # Add 'scripts' directory to sys.path
# sys.path.append(scripts_dir)

import transforms, utils, engine, train
from utils import collate_fn
from engine import train_one_epoch, evaluate


t = torch.cuda.get_device_properties(0).total_memory
print(t)
torch.cuda.empty_cache()

r = torch.cuda.memory_reserved(0)
print(r)
a = torch.cuda.memory_allocated(0)
print(a)
f = r-a  # free inside reserved

16908615680
517996544
503718400


In [18]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [19]:
net = UNet()
trained_unet = '/home/jc-merlab/Pictures/Data/trained_models/inpainting/14Dec_att_generative_was_YCB.pth'
net.load_state_dict(torch.load(trained_unet))
# discriminator = Discriminator()
# trained_discriminator = '/home/jc-merlab/Pictures/Data/trained_models/inpainting/14Dec_att_discrimiator_was_YCB.pth'
# discriminator.load_state_dict(torch.load(trained_discriminator))

<All keys matched successfully>

In [20]:
net.eval()
net.to(device)

image_transform = T.Compose([
    T.Resize((480, 640)),
    T.ToTensor(),
    T.Normalize(mean=[0.209555113170385, 0.22507974363977162, 0.20982026500023962], std=[0.20639409678896012, 0.19208633033458372, 0.20659148273508857]),
])


def denormalize(tensor, means, stds):
    means = torch.tensor(means).view(-1, 1, 1)
    stds = torch.tensor(stds).view(-1, 1, 1)

    if tensor.is_cuda:
        means = means.to(tensor.device)
        stds = stds.to(tensor.device)
    tensor = tensor * stds + means

    return tensor

def predict_frame(frame):
    # Convert frame to PIL Image to apply torchvision transforms
    frame = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    frame = image_transform(frame).unsqueeze(0)  # Add batch dimension
    frame = frame.cuda()  # or .cpu() if not using GPU
    with torch.no_grad():
        prediction = net(frame)
    # Optionally, reverse normalization or other transformations here
    # Convert the tensor to a numpy array for OpenCV to display or save
    prediction = denormalize(prediction, [0.209555113170385, 0.22507974363977162, 0.20982026500023962], [0.20639409678896012, 0.19208633033458372, 0.20659148273508857])

    prediction = prediction.squeeze().cpu().numpy()
    prediction = np.transpose(prediction, (1, 2, 0))  # Change from (C, H, W) to (H, W, C)
    prediction = (prediction * 255).astype(np.uint8)
    return prediction

# Open the video
cap = cv2.VideoCapture('/home/jc-merlab/Pictures/panda_data/images_for_occlusion/4/occ_test_4.avi')
# Prepare for saving the output video (if needed)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('/home/jc-merlab/Pictures/panda_data/images_for_occlusion/4/occ_test_4_op.avi', fourcc, 20.0, (640, 480))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    # Process and predict the frame
    output_frame = predict_frame(frame)
    # Save or display the frame
    out.write(output_frame)
    cv2.imshow('frame', output_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press Q to stop
        break

# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()